In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import codecs
import re
import sys,os
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_data = pd.read_csv('./train_clean.csv')
test_data = pd.read_csv('./test1_clean.csv')
train_len = len(train_data.review)
test_len = len(test_data.review)
labels = np.array(train_data.label.values.tolist())
labels
test_data.head()

,ID,review
0,1,Mulazmat ke bahali ke dua farma dein aur koe w...
1,2,Dua farma dain meri sehat k luay aur meray baa...
2,3,Tum khabees nahi kutti aurat ho 馃槀馃槀馃槀馃槇馃槇
3,4,Mery ly r Meri behn k ly dua kren k Allah mjhy...
4,5,Aur agar Imran peshawar mei banayega toh 100 p...


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(max_df=0.5,ngram_range=(1,3))
tfidf_matrix = tfidf_vec.fit_transform(train_data.review.astype('U').tolist()+test_data.review.astype('U').tolist())
train = tfidf_matrix[:train_len]
test = tfidf_matrix[train_len:]
# tfidf_array = pd.DataFrame({'review':tfidf_array})

In [4]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
LR = LogisticRegression()
MNB = MultinomialNB()
sfd = StratifiedKFold(n_splits=10)
sfd.get_n_splits(train, labels)
predicts = []
test_splits = [350,260,270,280,265,200,380,172,265,270]
count = 0
for train_index, test_index in sfd.split(train, labels):
    X_train = train[train_index]
    Y_train = labels[train_index]
    x_test = train[test_index]
    y_test = labels[test_index]
    MNB.fit(X_train, Y_train)
    print(roc_auc_score(y_test, MNB.predict_proba(x_test)[:, 1]))
    flag = test_splits[count]
    predicts.append(MNB.predict_proba(test[:flag])[:, 1])
    test = test[flag:]
    count += 1
predicts = [y for x in predicts for y in x]
len(predicts)

0.8793074163994045
0.841139570306237
0.8568923360590028
0.8607303190636524
0.8597633076799743
0.8393859227192559
0.8813832772166106
0.8367720398970399
0.8501045688545689
0.8675595238095238


2712

In [5]:
result = pd.DataFrame({
    'ID':list(range(1,test_len+1)),
    'Pred':predicts
})
result
result.to_csv('./res.csv',index=False)
print("Done!")

Done!
